In [3]:
import os
from dotenv import load_dotenv, find_dotenv

# 获取folder_path下所有文件路径，储存在file_paths里
file_paths = []
folder_path = '/workspaces/Xiaoti_Data_Interview_Preparation/knowledge_suanming'
for root, dirs, files in os.walk(folder_path):
    for file in files:
        file_path = os.path.join(root, file)
        file_paths.append(file_path)
print(file_paths[:3])
     

['/workspaces/Xiaoti_Data_Interview_Preparation/knowledge_suanming/010-奥修禅塔罗牌（版本一）.txt', '/workspaces/Xiaoti_Data_Interview_Preparation/knowledge_suanming/010-奥修禅塔罗牌[附图]（版本二）.txt (010-奥修禅塔罗牌[附图]（版本二）.txt) (Z-Library).txt', '/workspaces/Xiaoti_Data_Interview_Preparation/knowledge_suanming/清水玲子塔罗 解説書／G.タトテ研究所 （牌意部分）.pdf (清水玲子塔罗 解説書／G.タトテ研究所 （牌意部分）.pdf) (Z-Library).pdf']


In [10]:

from langchain_community.document_loaders import PyMuPDFLoader
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.document_loaders import TextLoader
# 遍历文件路径并把实例化的loader存放在loaders里
loaders = []

for file_path in file_paths:
    file_type = file_path.split('.')[-1]
    if file_type == 'pdf':
        loaders.append(PyMuPDFLoader(file_path))
    elif file_type == 'md':
        loaders.append(UnstructuredMarkdownLoader(file_path))
    elif file_type == 'txt':
        loaders.append(TextLoader(file_path, encoding='gbk'))

In [11]:
loaders

In [12]:
# 下载文件并存储到text
texts = []

for loader in loaders: texts.extend(loader.load())

RuntimeError: Error loading /workspaces/Xiaoti_Data_Interview_Preparation/knowledge_suanming/010-奥修禅塔罗牌[附图]（版本二）.txt (010-奥修禅塔罗牌[附图]（版本二）.txt) (Z-Library).txt